In [1]:
import numpy as np
import pandas as pd
import json

In [2]:
# given a df, a column to search, a search string, a column to edit, and a string to edit in,
# return the df with the column edited appropriately
def clean(df, find_str, find_col="city_ascii", edit_str=None, edit_col=None):
    if not edit_str:
        edit_str = find_str
    if not edit_col:
        edit_col = find_col
    df.loc[df[find_col].str.contains(find_str), edit_col] = edit_str
    return df

In [3]:
# read and merge all data tables
numbeo_beer_market = pd.read_table('numbeo_beer_market.txt', names=["rank", "city", "beer_market"], index_col="rank")
numbeo_beer_pub = pd.read_table("numbeo_beer_pub.txt", names=["rank", "city", "beer_pub"], index_col="rank")
numbeo_bread = pd.read_table('numbeo_bread.txt', names=["rank", "city", "bread"], index_col="rank")
numbeo_coffee = pd.read_table("numbeo_coffee.txt", names=['rank', 'city', 'coffee'], index_col="rank")

numbeo = pd.merge(numbeo_beer_market, numbeo_beer_pub, how="outer", on="city")
numbeo = pd.merge(numbeo, numbeo_bread, how="outer", on="city")
numbeo = pd.merge(numbeo, numbeo_coffee, how="outer", on="city")

In [4]:
# read in region, lat/lng, pop data
city_data = pd.read_csv('../city-data/world-cities-data.csv')
city_data['region'] = city_data['region'].str.title()

In [5]:
# corrections
city_data.loc[city_data.country.str.contains("Bahamas"), "country"] = "Bahamas"
city_data.loc[city_data.country.str.contains("Korea, South"), "country"] = "South Korea"

In [6]:
# create a separate country column
split = numbeo['city'].str.split(", ")
numbeo['country'] = split.str[-1]
numbeo['city'] = split.str[0]

In [7]:
# to align with city_data column names
numbeo = numbeo.rename(columns={"city": "city_ascii"})

In [8]:
# corrections
numbeo.city_ascii = numbeo.city_ascii.str.replace("Saint", "St.")
numbeo.loc[numbeo.city_ascii.str.contains("Petersburg") & numbeo.country.str.contains("Russia"), "city_ascii"] = "Saint Petersburg"

numbeo = clean(numbeo, find_str="Tel Aviv")
numbeo = clean(numbeo, find_str="Jeddah")
numbeo = clean(numbeo, find_str="Arhus", edit_str="Aarhus")
numbeo = clean(numbeo, find_str="Freiburg")
numbeo = clean(numbeo, find_str="Calicut")
numbeo = clean(numbeo, find_str="Lucknow")
# numbeo = clean(numbeo, find_str="Mumbai")
numbeo = clean(numbeo, find_str="Padova", edit_str="Padua")
numbeo = clean(numbeo, find_str="Astana")
numbeo = clean(numbeo, find_col="country", find_str="Kosovo")
numbeo = clean(numbeo, find_str="The Hague")
numbeo = clean(numbeo, find_str="Krakow")
numbeo = clean(numbeo, find_str="Zaragoza")
numbeo = clean(numbeo, find_str="Seville")
numbeo = clean(numbeo, find_str="Kyiv")
numbeo = clean(numbeo, find_str="Odessa")
numbeo = clean(numbeo, find_str="Novgorod", edit_str="Nizhniy Novgorod")


In [9]:
# load in expatistan data
with open("../expatistan/expatistan-data.json") as f:
    data = json.load(f)
expat = pd.json_normalize(data)
expat.columns = ['city_ascii', 'country', 'beer_market', 'bread', 'coffee', 'beer_pub']

expat.head()

,city_ascii,country,beer_market,bread,coffee,beer_pub
0,Kabul,Afghanistan,2.61,0.50,2.81,5.80
1,Tirana,Albania,1.06,0.78,1.56,3.58
2,Algiers,Algeria,0.88,0.23,2.29,2.10
3,Athens,Greece,1.53,0.91,4.72,5.27
4,Buenos Aires,Argentina,0.94,0.49,2.10,1.86


In [10]:
# merge expatistan and numbeo data
on_cols = ['city_ascii', 'country']
n_e = pd.merge(numbeo, expat, how="outer", on=on_cols, suffixes=("_n", "_e"))

In [11]:
# rows that have expatistan but not numbeo data
n_e[n_e["beer_pub_n"].isna() & n_e["bread_n"].isna() & n_e['coffee_n'].isna() & n_e["beer_market_n"].isna()]


,city_ascii,beer_market_n,beer_pub_n,bread_n,coffee_n,country,beer_market_e,bread_e,coffee_e,beer_pub_e
620,Santa Rosa,NaN,NaN,NaN,NaN,Argentina,1.15,0.40,1.16,2.24
621,Santa Clara,NaN,NaN,NaN,NaN,Cuba,1.22,1.50,3.53,5.91
622,St. Petersburg,NaN,NaN,NaN,NaN,United States,2.42,2.26,5.08,4.84


In [12]:
# rows that have numbeo but not expatistan data
n_e[n_e["beer_pub_e"].isna() & n_e["bread_e"].isna() & n_e['coffee_e'].isna() & n_e["beer_market_e"].isna()]


,city_ascii,beer_market_n,beer_pub_n,bread_n,coffee_n,country,beer_market_e,bread_e,coffee_e,beer_pub_e
50,Espoo,2.95,7.25,2.60,4.91,Finland,NaN,NaN,NaN,NaN
76,Brooklyn,2.72,7.00,3.56,4.32,United States,NaN,NaN,NaN,NaN
100,Brampton,2.53,4.33,1.83,2.65,Canada,NaN,NaN,NaN,NaN
101,St. Paul,2.53,5.00,2.75,4.25,United States,NaN,NaN,NaN,NaN
146,Sliema,2.22,3.62,1.70,2.93,Malta,NaN,NaN,NaN,NaN
147,Navi Mumbai,2.21,2.39,0.45,2.29,India,NaN,NaN,NaN,NaN
174,Derby,2.03,4.76,1.46,3.48,United Kingdom,NaN,NaN,NaN,NaN
178,Bali,2.01,2.47,1.57,2.11,Indonesia,NaN,NaN,NaN,NaN
181,Chennai,2.00,2.05,0.50,1.99,India,NaN,NaN,NaN,NaN
203,Lund,1.88,7.74,3.01,4.33,Sweden,NaN,NaN,NaN,NaN


In [13]:
# corrections prior to merging with city_data
corrections = [['Newcastle', "Newcastle"],
              ["Penang", "George Town"],
              ['Heraklion', 'Irakleio'],
              ['Ahmedabad', 'Ahmadabad'],
              ['Patras', 'Patra'],
              ['Pattaya', 'Phatthaya'],
              ['Visakhapatnam', 'Vishakhapatnam'],
              ['Hanover', 'Hannover'],
              ['Yangon', 'Rangoon'],
              ['Rostov', 'Rostov'],
              ["Goa", "Panaji"],
              ['Ain', "Al `Ayn"],
              ["Santa Cruz", "Santa Cruz"],
              ["Ajman", "`Ajman"],
              ['Chittagong', 'Chattogram'],
              ['Macao', 'Macau'],
              ['Marsa', 'La Marsa'],
              ['Astana', 'Nur-Sultan'],
              ["Seville", "Sevilla"]]

for find, edit in corrections:
    n_e = clean(n_e, find_str=find, edit_str=edit)

In [14]:
n_e.loc[n_e.city_ascii.str.contains("Rangoon"), "country"] = "Burma"
n_e.loc[n_e.city_ascii.str.contains("Macau"), "country"] = "Macau"

n_e.loc[n_e.country.str.contains("Cz"), "country"] = "Czechia"
n_e.loc[n_e.country.str.contains("Mace"), "country"] = "Macedonia"
n_e.loc[n_e.country.str.contains("Ivory"), "country"] = "Côte D’Ivoire"
n_e.loc[n_e.country.str.contains("Kosovo"), "country"] = "Kosovo"

In [15]:
# TODO are these still used?
cols = ["city_ascii", 'country','region', 'latitude', 'longitude', 'population', 'admin_name']
admin_cols = ['city_ascii', 
              'admin_name', 
              'country', 
              'region', 
              'latitude', 
              'longitude', 
              'population',
              'beer_market_n', 
              'bread_n', 
              'coffee_n', 
              'beer_pub_n', 
              'beer_market_e', 
              'bread_e', 
              'coffee_e', 
              'beer_pub_e', 
]

In [16]:
# for US cities, generate a list of unabbreviated state names
def state_unabbr(state):
    abbr = pd.read_table("../city-data/us-state-abbr.tsv", names=['abbr', 'name'], header=0)
    name = abbr.loc[abbr['abbr'] == state, 'name']
    if len(name) > 0:
        return name.values[0]
    else:
        return np.nan

states = []
for i in split:
    if i[-1] == "United States":
        states.append(state_unabbr(i[1]))
        
states.append("Florida")

In [17]:
n_e[n_e['city_ascii'] == 'St. Louis']

,city_ascii,beer_market_n,beer_pub_n,bread_n,coffee_n,country,beer_market_e,bread_e,coffee_e,beer_pub_e
127,St. Louis,2.32,4.5,1.99,4.53,United States,2.37,1.95,5.42,5.72


In [18]:
# separate numbeo into US and non-US cities
# for US cities, add states under admin_name, then merge with city_data latitude, longitude, region, and population
numbeo_us = n_e[n_e['country'] == 'United States']
numbeo_us["admin_name"] = states

numbeo_us = pd.merge(numbeo_us, city_data[cols], how='left', on=['city_ascii', 'country', 'admin_name'])
numbeo_us = numbeo_us.reindex(columns=admin_cols)

/Users/christalee/Library/Python/3.6/lib/python/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [19]:
len(numbeo_us) == len(n_e[n_e['country'] == 'United States'])

True

In [20]:
numbeo_us.tail()

,city_ascii,admin_name,country,region,latitude,longitude,population,beer_market_n,bread_n,coffee_n,beer_pub_n,beer_market_e,bread_e,coffee_e,beer_pub_e
105,Riverside,California,United States,North America,33.9381,-117.3948,2107852.0,NaN,2.65,NaN,NaN,2.48,2.53,4.74,5.68
106,Henderson,Nevada,United States,North America,36.0133,-115.0380,320189.0,NaN,2.48,3.85,NaN,NaN,NaN,NaN,NaN
107,Lincoln,Nebraska,United States,North America,40.8090,-96.6788,289490.0,NaN,2.40,3.55,NaN,2.68,1.53,4.86,4.56
108,Mesa,Arizona,United States,North America,33.4017,-111.7180,518012.0,NaN,NaN,4.40,NaN,NaN,NaN,NaN,NaN
109,St. Petersburg,Florida,United States,North America,27.7931,-82.6652,265351.0,NaN,NaN,NaN,NaN,2.42,2.26,5.08,4.84


In [21]:
# for non-US cities, don't match on admin_name
numbeo_globe = n_e[n_e['country'] != "United States"]

numbeo_globe = pd.merge(numbeo_globe, city_data[cols], how='left', on=['city_ascii', 'country'])
numbeo_globe = numbeo_globe.reindex(columns=admin_cols)

In [22]:
# rows where city & country had more than one match in city_data
x = n_e[n_e['country'] != "United States"]
for city, country in zip(x['city_ascii'], x['country']):
    y = numbeo_globe.loc[(numbeo_globe.city_ascii == city) & (numbeo_globe.country == country)]
    if len(y) != 1:
        print(y)

   city_ascii admin_name country   region  latitude  longitude  population  \
75    Uppsala    Uppsala  Sweden  Europe    59.8498    17.6389    164535.0   
76    Uppsala  Stockholm  Sweden  Europe    59.8601    17.6400    133117.0   

    beer_market_n  bread_n  coffee_n  beer_pub_n  beer_market_e  bread_e  \
75           2.47     2.92      4.11        8.34           1.95     1.52   
76           2.47     2.92      4.11        8.34           1.95     1.52   

    coffee_e  beer_pub_e  
75      4.48         7.0  
76      4.48         7.0  
    city_ascii admin_name country          region  latitude  longitude  \
132    Windsor    Ontario  Canada  North America    42.2833      -83.0   
133    Windsor     Quebec  Canada  North America    45.5667      -72.0   

     population  beer_market_n  bread_n  coffee_n  beer_pub_n  beer_market_e  \
132    276165.0           2.01     2.15      3.27        4.72           2.28   
133      5367.0           2.01     2.15      3.27        4.72           

In [23]:
# dedupe those rows
numbeo_globe = numbeo_globe.drop(index=[76, 133, 309, 388])

In [24]:
def update_row(wrong_city, right_data):
    right_data["city"] = wrong_city
    
    y = lost[lost['city_ascii'] == wrong_city]
    y = y.drop(columns=['admin_name', 'region', 'latitude', 'longitude', 'population'])
    
    merge_cols = ['city_ascii', 'admin_name', 'region', 'latitude', 'longitude', 'population', 'city']
    z = pd.merge(y, right_data[merge_cols], how='left', left_on=['city_ascii'], right_on=['city'])
    z = z.drop(columns=['city_ascii_x', 'city'])
    z = z.rename(columns={'city_ascii_y': 'city_ascii'})
    z = z.reindex(columns=admin_cols)
    
    return z

In [25]:
lost = numbeo_globe[np.isnan(numbeo_globe['latitude'])]

# for i in range(len(lost)):
#     row = lost.iloc[i]
#     city = row['city_ascii']
#     country = row['country']
#     index = row.name
#     x = city_data[city_data.city.str.contains(city.strip("()")[-4:]) & (city_data['country'] == country)]
#     y = city_data[city_data.city.str.contains(city.strip("()")[:4]) & (city_data['country'] == country)]
#     if len(x) == 1:
#         print(city, x)
# #         lost[lost.index == index] = update_row(city, x).values
#     elif len(y) == 1:
#         print(city, y)
# #         lost[lost.index == index] = update_row(city, y).values

In [26]:
lost

,city_ascii,admin_name,country,region,latitude,longitude,population,beer_market_n,bread_n,coffee_n,beer_pub_n,beer_market_e,bread_e,coffee_e,beer_pub_e
28,Tel Aviv,NaN,Israel,NaN,NaN,NaN,NaN,3.29,2.39,4.21,9.12,2.73,1.53,4.31,8.00
95,Cardiff,NaN,United Kingdom,NaN,NaN,NaN,NaN,2.28,1.52,3.71,4.76,2.61,1.26,3.52,4.69
106,Navi Mumbai,NaN,India,NaN,NaN,NaN,NaN,2.21,0.45,2.29,2.39,NaN,NaN,NaN,NaN
134,Bali,NaN,Indonesia,NaN,NaN,NaN,NaN,2.01,1.57,2.11,2.47,NaN,NaN,NaN,NaN
152,Lund,NaN,Sweden,NaN,NaN,NaN,NaN,1.88,3.01,4.33,7.74,NaN,NaN,NaN,NaN
164,Vijayawada,NaN,India,NaN,NaN,NaN,NaN,1.79,0.54,1.78,1.30,1.90,0.59,1.29,2.68
170,Noida,NaN,India,NaN,NaN,NaN,NaN,1.77,0.48,1.97,1.91,NaN,NaN,NaN,NaN
193,Bhubaneswar,NaN,India,NaN,NaN,NaN,NaN,1.63,0.54,1.09,2.05,1.60,0.45,2.35,2.19
203,Padua,NaN,Italy,NaN,NaN,NaN,NaN,1.55,1.77,1.70,4.83,1.26,1.88,1.68,5.73
208,Makati,NaN,Philippines,NaN,NaN,NaN,NaN,1.52,NaN,2.78,1.46,NaN,NaN,NaN,NaN


In [27]:
import qwikidata.sparql

def get_city_wikidata(city, country):
    query = """
    SELECT ?city ?cityLabel ?country ?countryLabel ?population
    WHERE
    {
      ?city rdfs:label '%s'@en.
      ?city wdt:P1082 ?population.
      ?city wdt:P17 ?country.
      ?city rdfs:label ?cityLabel.
      ?country rdfs:label ?countryLabel.
      FILTER(LANG(?cityLabel) = "en").
      FILTER(LANG(?countryLabel) = "en").
      FILTER(CONTAINS(?countryLabel, "%s")).
    }
    """ % (city, country)

    res = qwikidata.sparql.return_sparql_query_results(query)
    out = res['results']['bindings'][0]
    return out

In [28]:
import geocoder
import time
from requests import Session

with Session() as s:
    for row in lost[np.isnan(lost.latitude)].values:
        city = row[0]
        country = row[2]
        try:
            latlng = geocoder.osm(city + ", " + country, session=s).latlng
            if latlng:
#             print(city, country, latlng)
                lost.loc[lost.city_ascii == city, 'latitude'] = latlng[0]
                lost.loc[lost.city_ascii == city, "longitude"] = latlng[1]
            
            wikidata = get_city_wikidata(city, country)
            lost.loc[lost.city_ascii == city, "population"] = int(wikidata['population']['value'])
        except:
            pass
        time.sleep(1)


/Users/christalee/Library/Python/3.6/lib/python/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [29]:
for i in range(len(lost)):
    row = lost.iloc[i]
    index = row.name
    numbeo_globe[numbeo_globe.index == index] = row.values

In [30]:
n_e_clean = pd.concat([numbeo_us, numbeo_globe])
len(n_e) == len(n_e_clean)

True

In [31]:
numbeo_globe.head(25)

,city_ascii,admin_name,country,region,latitude,longitude,population,beer_market_n,bread_n,coffee_n,beer_pub_n,beer_market_e,bread_e,coffee_e,beer_pub_e
0,Doha,Ad Dawḩah,Qatar,Middle East,25.3000,51.5333,1312947.0,8.51,1.45,4.63,13.35,4.82,1.18,7.00,13.00
1,Amman,Al ‘Āşimah,Jordan,Middle East,31.9500,35.9333,4007526.0,5.30,0.41,4.14,7.05,4.81,0.49,4.37,8.00
2,Newcastle,New South Wales,Australia,Asia & Pacific,-32.9167,151.7500,322278.0,4.70,1.67,3.52,6.15,3.65,1.38,3.44,5.26
3,Muscat,Masqaţ,Oman,Middle East,23.6139,58.5922,1421409.0,4.66,1.08,4.61,9.09,3.93,0.90,5.07,9.00
4,Melbourne,Victoria,Australia,Asia & Pacific,-37.8136,144.9631,5078193.0,4.57,2.34,3.40,7.69,4.51,2.06,3.38,8.00
5,Adelaide,South Australia,Australia,Asia & Pacific,-34.9289,138.6011,1345777.0,4.54,2.10,3.31,6.15,4.21,2.06,3.47,6.00
6,Trondheim,Sør-Trøndelag,Norway,Europe,63.4400,10.4000,183378.0,4.52,3.66,5.21,11.67,3.98,2.84,5.18,10.00
7,Brisbane,Queensland,Australia,Asia & Pacific,-27.4678,153.0281,2514184.0,4.50,2.12,3.55,6.15,4.48,1.75,3.77,7.00
8,Canberra,Australian Capital Territory,Australia,Asia & Pacific,-35.2931,149.1269,426704.0,4.26,2.57,3.45,6.15,3.70,1.89,3.51,7.00
9,Sydney,New South Wales,Australia,Asia & Pacific,-33.8650,151.2094,5312163.0,4.09,2.05,3.27,6.15,4.22,1.81,3.32,6.00


In [32]:
# numbeo.to_csv("numbeo.csv")

n_e_clean.index = list(range(len(n_e_clean)))
n_e_clean.to_csv("n_e_clean.csv")
n_e_clean.to_json("n_e_clean.json")